In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
initialBlocks = {
    0: " . ", 1: " . ", 2: " . ",
    3: " . ", 4: " . ", 5: " . ",
    6: " D ", 7: " A ", 8: " . ",
    9: " C ", 10: " B ", 11: " . "
}
finalBlocks = {
    0: " . ", 1: " . ", 2: " . ",
    3: " . ", 4: " . ", 5: " . ",
    6: " C ", 7: " B ", 8: " . ",
    9: " A ", 10: " D ", 11: " . "
}
initialBinaryVector = [0] * 24

for i in range(12):
    if initialBlocks[i] == " . ":
        initialBinaryVector[i + 12] = 1  # Add gap
    else:
        initialBinaryVector[i] = 1  # Add block

In [ ]:
# Class for nodes
class Node:
    def __init__(self, state, blocks, parent=None):
        self.state = state
        self.blocks = blocks
        self.parent = parent

In [ ]:
# Function to evaluate if an action is possible
def validMove(currentState, i, j):
    precondition = [0] * 24
    precondition[i] = 1  # Block in the initial position
    precondition[j + 12] =1  # Gap in the final position
    if i - 3 >= 0:
        precondition[i + 9] = 1  # Gap above the initial position
    if i+ 3 <= 11:
        precondition[i + 3]= 1  # Block below the initial position
    if j - 3 >= 0:
        precondition[j +9] = 1  # Gap above the final position
    if j +3 <=11:
        precondition[j+ 3]= 1  # Block below the final position
    return j !=i-3 and all((currentState[k] & precondition[k])== precondition[k] for k in range(24))

# Update the block dictionary for each possible move
def updateBlocks(blocks, i, j):
    blocks = blocks
    blocks[j] = blocks[i]
    blocks[i] = " . "
    return blocks

# Removing the block in the initial position
def moveBlock(i, j, currentState):
    removal = [1] * 24
    removal[i] = 0 # Remove block in the initial position
    removal[j + 12] = 0 # Remove gap in the final position
    currentState = [x & y for x, y in zip(currentState, removal)]
    addition = [0] * 24
    addition[j] = 1 # Add block in the final position
    addition[i + 12] = 1 # Add gap in the initial position
    return [x | y for x, y in zip(currentState, addition)]

# Generate the children of a node
def generate_children(node):
    for i in range(12):
        for j in range(12):
            if validMove(node.state, i, j):
                childState = moveBlock(i, j, node.state)
                childBlocks = updateBlocks(node.blocks, i, j)
                child_tuple = tuple(childBlocks.items())
                if child_tuple not in visited:
                    visited.add(child_tuple)
                    yield Node(childState, childBlocks, node)
                    
# Create the path from the root node to the final node
def createPath(node):
    path = []
    while node:
        path.insert(0, node)
        node = node.parent
    return path

# Breadth-first search function using a stack
def breadth_first_search():
    queue = [root]
    visited = set()
    while queue:
        current_node = queue.pop(0)
        visited.add(tuple(current_node.blocks.items()))
        if current_node.blocks == finalBlocks:
            return current_node
        for child in generate_children(current_node):
            if tuple(child.blocks.items()) not in visited:
                queue.append(child)
    return None

# Function to convert blocks into a 4x3 matrix
def convert_to_matrix(blocks):
    matrix = [[""]*3 for _ in range(4)]
    for i in range(4):
        for j in range(3):
            matrix[i][j] = blocks.get(i * 3 + j, "")
    return matrix

In [ ]:
visited = set()  # Visited nodes
visited.add(tuple(initialBlocks.items()))
root = Node(initialBinaryVector, initialBlocks)  # Root node

# Function to display a state in a grid
def display_state(matrix):
    for row in matrix:
        print("".join(row))
    print()
# Execute the breadth-first search
solution = breadth_first_search()

# Visualize the sequence of states if a solution is found
if solution:
    path = createPath(solution)
    for node in path:
        matrix = convert_to_matrix(node.blocks)
        display_state(matrix)



 .  .  . 
 .  .  . 
 D  A  . 
 C  B  . 

 .  .  . 
 .  .  . 
 .  A  . 
 C  B  D 

 .  .  . 
 .  .  . 
 .  A  C 
 .  B  D 

 .  .  . 
 .  .  . 
 .  .  C 
 A  B  D 

 .  .  . 
 .  .  . 
 C  .  . 
 A  B  D 

 .  .  . 
 B  .  . 
 C  .  . 
 A  .  D 

 .  .  . 
 B  .  . 
 C  .  . 
 A  D  . 

 .  .  . 
 .  .  . 
 C  B  . 
 A  D  . 

